Import thư viện

In [1]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine, text
from pandas import Int64Dtype
from dotenv import load_dotenv
import os

Kết nối đến DB MSSQL SERVER sử dụng sqlalchemy

In [7]:

server = 'HuyThai\SQLEXPRESS'
database = 'Thai1'
dwh = 'ThaiDWH'
username = os.environ["user"]
password = os.environ["pass"]
driver = 'ODBC Driver 17 for SQL Server' 

# Tạo URL kết nối SQLAlchemy
url = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'
url_dwh = f'mssql+pyodbc://{username}:{password}@{server}/{dwh}?driver={driver}'
# Tạo đối tượng Engine
engine = create_engine(url)
engine_dwh = create_engine(url_dwh)
# Tạo connection
conn_source = engine.connect()
conn_dwh = engine_dwh.connect()



Extract Data từ database vào pandas dataframe

In [3]:
def extract():
    customers = pd.read_sql(text('SELECT * FROM Customers'), conn_source)
    products = pd.read_sql(text('SELECT * FROM Product'), conn_source, dtype={'ProductSubcategoryID': Int64Dtype()})
    order_header = pd.read_sql(text('SELECT * FROM OrderHeader'), conn_source)
    order_detail = pd.read_sql(text('SELECT * FROM OrderDetail'), conn_source)
    product_subcategory = pd.read_sql(text('SELECT * FROM ProductSubCategory'), conn_source)
    product_category = pd.read_sql(text('SELECT * FROM ProductCategory'), conn_source)
    return customers, products, order_header, order_detail, product_subcategory, product_category

In [4]:
customers, products, order_header, order_detail, product_subcategory, product_category = extract()

In [6]:
customers.head()

,CustomerID,AccountNumber,FirstName,MiddleName,LastName
0,1,AW00000001,None,None,None
1,2,AW00000002,None,None,None
2,3,AW00000003,None,None,None
3,4,AW00000004,None,None,None
4,5,AW00000005,None,None,None


In [7]:
products.head()

,ProductID,Name,Color,ListPrice,Size,ProductSubcategoryID
0,1,Adjustable Race,None,0.0,None,<NA>
1,2,Bearing Ball,None,0.0,None,<NA>
2,3,BB Ball Bearing,None,0.0,None,<NA>
3,4,Headset Ball Bearings,None,0.0,None,<NA>
4,316,Blade,None,0.0,None,<NA>


In [8]:
order_header.head()

,SalesOrderID,OrderDate,ShipDate,SalesOrderNumber,CustomerID,SubTotal
0,43659,2011-05-31,2011-06-07,SO43659,29825,205656206.0
1,43660,2011-05-31,2011-06-07,SO43660,29672,12942529.0
2,43661,2011-05-31,2011-06-07,SO43661,29734,327264786.0
3,43662,2011-05-31,2011-06-07,SO43662,29994,288325289.0
4,43663,2011-05-31,2011-06-07,SO43663,29565,4194589.0


In [9]:
order_detail.head()

,SalesOrderID,SalesOrderDetailID,ProductID,OrderQty,UnitPrice,UnitPriceDiscount
0,43659,1,776,1,2024994.0,0.0
1,43659,2,777,3,2024994.0,0.0
2,43659,3,778,1,2024994.0,0.0
3,43659,4,771,1,2039994.0,0.0
4,43659,5,772,1,2039994.0,0.0


In [5]:
product_category.head()

,ProductCategoryID,Name
0,1,Bikes
1,2,Components
2,3,Clothing
3,4,Accessories


In [11]:
product_subcategory.head()

,ProductSubcategoryID,ProductCategoryID,Name
0,1,1,Mountain Bikes
1,2,1,Road Bikes
2,3,1,Touring Bikes
3,4,2,Handlebars
4,5,2,Bottom Brackets


Transform data

In [12]:
# Tạo các bảng dimension
dim_customer = customers[['CustomerID', 'AccountNumber', 'FirstName', 'MiddleName', 'LastName']].copy()
dim_product = products[['ProductID', 'Name', 'Color', 'ListPrice', 'Size', 'ProductSubcategoryID']].copy()
dim_product_subcategory = product_subcategory[['ProductSubcategoryID', 'Name', 'ProductCategoryID']].copy()
dim_product_category = product_category[['ProductCategoryID', 'Name']].copy()
dim_order_header = order_header[['SalesOrderID','ShipDate', 'SalesOrderNumber', 'SubTotal']].copy()

In [13]:
# Tạo bảng dim date
order_header['OrderDate'] = pd.to_datetime(order_header['OrderDate'])
dim_date = order_header[['OrderDate']].drop_duplicates().copy()
dim_date['DateID'] = dim_date['OrderDate'].dt.strftime('%Y%m%d').astype(int)
dim_date['Day'] = dim_date['OrderDate'].dt.day
dim_date['Month'] = dim_date['OrderDate'].dt.month
dim_date['Quarter'] = dim_date['OrderDate'].dt.quarter
dim_date['Year'] = dim_date['OrderDate'].dt.year
dim_date['Weekday'] = dim_date['OrderDate'].dt.weekday

In [14]:
dim_date.head()

,OrderDate,DateID,Day,Month,Quarter,Year,Weekday
0,2011-05-31,20110531,31,5,2,2011,1
43,2011-06-01,20110601,1,6,2,2011,2
47,2011-06-02,20110602,2,6,2,2011,3
52,2011-06-03,20110603,3,6,2,2011,4
54,2011-06-04,20110604,4,6,2,2011,5


In [25]:
# Tạo bảng fact sales
fact_sales = order_detail.merge(order_header, on='SalesOrderID')
fact_sales['TotalAmount'] = fact_sales['OrderQty'] * (fact_sales['UnitPrice'] - fact_sales['UnitPriceDiscount'])
fact_sales = fact_sales[['SalesOrderDetailID', 'SalesOrderID', 'CustomerID', 'ProductID', 'OrderDate', 'OrderQty', 'UnitPrice', 'UnitPriceDiscount', 'TotalAmount']]
fact_sales = fact_sales.merge(dim_date, left_on='OrderDate', right_on='OrderDate')
fact_sales = fact_sales[['SalesOrderDetailID', 'SalesOrderID', 'CustomerID', 'ProductID', 'DateID', 'OrderQty', 'UnitPrice', 'UnitPriceDiscount', 'TotalAmount']]

In [26]:
fact_sales = fact_sales.merge(products, on='ProductID')
fact_sales = fact_sales.merge(product_subcategory, on='ProductSubcategoryID')
fact_sales = fact_sales.merge(product_category, on='ProductCategoryID')

In [27]:
fact_sales = fact_sales[['SalesOrderDetailID', 'SalesOrderID', 'CustomerID', 'ProductID', 'DateID', 'ProductSubcategoryID', 'ProductCategoryID', 'OrderQty', 'UnitPrice', 'UnitPriceDiscount', 'TotalAmount']]

In [28]:
fact_sales

,SalesOrderDetailID,SalesOrderID,CustomerID,ProductID,DateID,ProductSubcategoryID,ProductCategoryID,OrderQty,UnitPrice,UnitPriceDiscount,TotalAmount
0,1,43659,29825,776,20110531,1,1,1,2024994.0,0.0,2024994.0
1,25,43661,29734,776,20110531,1,1,4,2024994.0,0.0,8099976.0
2,70,43665,29580,776,20110531,1,1,1,2024994.0,0.0,2024994.0
3,114,43670,29566,776,20110531,1,1,1,2024994.0,0.0,2024994.0
4,127,43672,30067,776,20110531,1,1,2,2024994.0,0.0,4049988.0
...,...,...,...,...,...,...,...,...,...,...,...
121312,113253,71936,30050,994,20140501,5,2,3,32394.0,0.0,97182.0
121313,113359,71943,29931,994,20140501,5,2,3,32394.0,0.0,97182.0
121314,113393,71945,30092,994,20140501,5,2,1,32394.0,0.0,32394.0
121315,113420,71947,29551,994,20140501,5,2,1,32394.0,0.0,32394.0


In [29]:
fact_sales.head()

,SalesOrderDetailID,SalesOrderID,CustomerID,ProductID,DateID,ProductSubcategoryID,ProductCategoryID,OrderQty,UnitPrice,UnitPriceDiscount,TotalAmount
0,1,43659,29825,776,20110531,1,1,1,2024994.0,0.0,2024994.0
1,25,43661,29734,776,20110531,1,1,4,2024994.0,0.0,8099976.0
2,70,43665,29580,776,20110531,1,1,1,2024994.0,0.0,2024994.0
3,114,43670,29566,776,20110531,1,1,1,2024994.0,0.0,2024994.0
4,127,43672,30067,776,20110531,1,1,2,2024994.0,0.0,4049988.0


Load data vào db

In [17]:
dim_customer.to_sql('DimCustomer', conn_dwh, if_exists='replace', index=False, method='multi', chunksize=50)

19820

In [18]:
dim_product.to_sql('DimProduct', conn_dwh, if_exists='replace', index=False, method='multi', chunksize=50)

504

In [19]:
dim_product_subcategory.to_sql('DimProductSubcategory', conn_dwh, if_exists='replace', index=False, method='multi', chunksize=50)

37

In [10]:
dim_product_category.to_sql('DimProductCategory', conn_dwh, if_exists='replace', index=False, method='multi', chunksize=50)

4

In [9]:
dim_order_header.to_sql('DimOrderHeader', conn_dwh, if_exists='replace', index=False, method='multi', chunksize=50)

31465

In [22]:
dim_date.to_sql('DimDate', conn_dwh, if_exists='replace', index=False, method='multi', chunksize=50)

1124

In [30]:
fact_sales.to_sql('FactSales', conn_dwh, if_exists='replace', index=False, method='multi', chunksize=50)

121317